In [1]:
import xarray
import numpy as np
import pandas
import h5py
import datetime
from ulmo.utils import image_utils
from ulmo.plotting import plotting
import matplotlib.pyplot as plt
import seaborn as sns
import os

/Users/rachelstumpf/ulmo/ulmo/utils/image_utils.py:8: UserWarning: healpy not installed.  Not all functions will work!
  warnings.warn("healpy not installed.  Not all functions will work!")


In [2]:
# check extraction tables

table_2012 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2012.parquet')
table_2013 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2013.parquet')
table_2014 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2014.parquet')
table_2015 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2015.parquet')
table_2016 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2016.parquet')
table_2017 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2017.parquet')
table_2018 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2018.parquet')
table_2019 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2019.parquet')
table_2020 = pandas.read_parquet('/Users/rachelstumpf/Desktop/parquet/SST_L3S_VIIRS_2020.parquet')

table_all = pandas.concat([table_2012, table_2013, table_2014, table_2015,
                        table_2016, table_2017, table_2018, table_2019, table_2020])
table_all.reset_index(drop=True, inplace=True)
table_all = table_all.dropna(subset=['LL'])

In [29]:
table_all.loc[:,'local_timedelta'] = pandas.to_timedelta(table_all.loc[:,'lon'] / 15, unit='H')
table_all.loc[:,'local_suntime'] = table_all.loc[:,'VIIRS_datetime'] + table_all.loc[:,'local_timedelta']
normalized_date = table_all.loc[:,'local_suntime'].dt.normalize()
time_component_timedelta = pandas.to_timedelta(table_all.loc[:,'local_suntime'].dt.hour, unit='h') + pandas.to_timedelta(table_all['local_suntime'].dt.minute, unit='m') + pandas.to_timedelta(table_all.loc[:,'local_suntime'].dt.second, unit='s')

table_all.loc[:,'L2_local_suntime'] = normalized_date + time_component_timedelta
table_all['L2_local_suntime_hours'] = table_all['L2_local_suntime'].dt.hour + table_all['L2_local_suntime'].dt.minute / 60 + table_all['L2_local_suntime'].dt.second / 3600

In [30]:
table_all.loc[:,'LL_diff'] = table_all.loc[:,'VIIRS_LL'] - table_all.loc[:,'LL']

In [31]:
nighttime_start = 22.0
nighttime_end = 6.0

nighttime_cutouts = table_all[
    (table_all['L2_local_suntime_hours'] >= nighttime_start) |
    (table_all['L2_local_suntime_hours'] <= nighttime_end)
]
daytime_cutouts = table_all[
    (table_all['L2_local_suntime_hours'] <= nighttime_start) &
    (table_all['L2_local_suntime_hours'] >= nighttime_end)
]

In [32]:
filtered_data = nighttime_cutouts[nighttime_cutouts['LL_diff'] >= 100]
random_sample = filtered_data.sample(n=10, random_state=42)
pandas.set_option('display.max_colwidth', None)
table = pandas.DataFrame(random_sample[['VIIRS_pp_file', "VIIRS_pp_idx", 'VIIRS_datetime', 'VIIRS_filename', 'VIIRS_row', 'VIIRS_col', 'row', 'col', 'lat', 'lon']])
table.to_csv('higher_VIIRS_LL.csv', index=False)

In [33]:
filtered_data = nighttime_cutouts[nighttime_cutouts['LL_diff'] <= -100]
random_sample = filtered_data.sample(n=10, random_state=42)
pandas.set_option('display.max_colwidth', None)
table = pandas.DataFrame(random_sample[['VIIRS_pp_file', "VIIRS_pp_idx", 'VIIRS_datetime', 'VIIRS_filename', 'VIIRS_row', 'VIIRS_col', 'row', 'col', 'lat', 'lon']])
table.to_csv('higher_L3S_LL.csv', index=False)